Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)


def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()

text = read_data(filename)
print('\nData size %d' % len(text))


valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print('\ntrain_size : ',train_size, train_text[:64])
print('\nvalid_size : ',valid_size, valid_text[:64])


vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
bi_vocabulary_size = vocabulary_size * vocabulary_size
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('\nUnexpected character: %s' % char)
        return 0

d1, d2 = {}, {}
for i in np.arange(vocabulary_size):
  for j in np.arange(vocabulary_size):
    v = i*vocabulary_size + j
    d1[(i,j)] = v
    d2[v] = (i,j)

def bi_char2id(char1, char2):
    id1 = char2id(char1)
    id2 = char2id(char2)
    return d1[(id1,id2)]

def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

def bi_id2char(dictid):
    (id1,id2) = d2[dictid]
    return id2char(id1)+id2char(id2)

print('\n version mono-gram :')
print('\n', char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print('\n', id2char(1), id2char(26), id2char(0))

print('\n version bi-gram :')
print('\n', bi_char2id('a','a'), bi_char2id('z','z'), bi_char2id(' ',' '), bi_char2id('ï','a'))
print('\n', bi_id2char(1), bi_id2char(32), bi_id2char(0))

Found and verified text8.zip

Data size 100000000

train_size :  99999000 ons anarchists advocate social relations based upon voluntary as

valid_size :  1000  anarchism originated as a term of abuse first used against earl

 version mono-gram :

Unexpected character: ï

 1 26 0 0

 a z  

 version bi-gram :

Unexpected character: ï

 28 728 0 1

  a ae   


In [2]:
batch_size = 64 # number of elements (elements == string) for each batch
num_unrollings = 10 # number of char for each batch element (ie each string) = num_unrollings+1 = 11


############################### version mono-gram ###############################

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones. """ 
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

############################### version bi-gram ###############################

class bi_BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)] # vector of the batch_size cursors positions in text
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, bi_vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, bi_char2id(self._text[self._cursor[b]], self._text[self._cursor[b]+1])] = 1.0
            self._cursor[b] = (self._cursor[b] + 2) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones. """ 
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def bi_characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [bi_id2char(c) for c in np.argmax(probabilities, 1)]

def bi_batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, bi_characters(b))]
    return s

bi_train_batches = bi_BatchGenerator(train_text, batch_size, num_unrollings)
bi_valid_batches = bi_BatchGenerator(valid_text, 1, 1)

#################################################################################

print('\n bi 1 :',bi_batches2string(bi_train_batches.next()))
print('\n bi 2 :',bi_batches2string(bi_train_batches.next()))
print('\n bi 3 :',bi_batches2string(bi_train_batches.next()))
print('\n one 1 :',batches2string(train_batches.next()))
print('\n one 2 :',batches2string(train_batches.next()))
print('\n one 3 :',batches2string(train_batches.next()))


 bi 1 : ['ons anarchists advocat', 'when military governme', 'lleria arches national', ' abbeys and monasterie', 'married urraca princes', 'hel and richard baer h', 'y and liturgical langu', 'ay opened for passenge', 'tion from the national', 'migration took place d', 'new york other well kn', 'he boeing seven six se', 'e listed with a gloss ', 'eber has probably been', 'o be made to recognize', 'yer who received the f', 'ore significant than i', 'a fierce critic of the', ' two six eight in sign', 'aristotle s uncaused c', 'ity can be lost as in ', ' and intracellular ice', 'tion of the size of th', 'dy to pass him a stick', 'f certain drugs confus', 'at it will take to com', 'e convince the priest ', 'ent told him to name i', 'ampaign and barred att', 'rver side standard for', 'ious texts such as eso', 'o capitalize on the gr', 'a duplicate of the ori', 'gh ann es d hiver one ', 'ine january eight marc', 'ross zero the lead cha', 'cal theories classical', 'ast instance the non g', ' 

In [3]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities. """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

############################### version mono-gram ###############################

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p


def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

############################### version bi-gram ###############################

def bi_sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, bi_vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p


def bi_random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, bi_vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [ ]:
num_nodes = 64 # number of neurons of the hidden layer (can be seen as the number of memory cells)
# vocabulary_size = 27 par construction
# batch_size = 64 # number of elements (elements == string) for each batch
# num_unrollings = 10 # number of char for each batch element (ie each string) = num_unrollings+1 = 11

graph = tf.Graph()
with graph.as_default():

    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))

    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings] # the first num_unrollings list elements
    train_labels = train_data[1:]  # the last num_unrollings list elements (labels are inputs shifted by one time step).

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( logits, tf.concat(0, train_labels) ) )

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay( 10.0, global_step, 5000, 0.1, staircase=True )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients( zip(gradients, v), global_step=global_step )

    # Predictions.
    train_prediction = tf.nn.softmax(logits)

    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group( saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell( sample_input, saved_sample_output, saved_sample_state )
    with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))



num_steps = 7001
num_steps = 1
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        # batch_size = 64 # number of elements (elements == string) for each batch
        # num_unrollings = 10 # number of char for each batch element (ie each string) = num_unrollings+1 = 11
        batches = train_batches.next() # (batch_size=64) strings of (num_unrollings=10) characters each
        #print('\nlen(batches) : ', len(batches))
        #print('\nlen(batches[0]) : ', len(batches[0]))
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i] # batches[i] is an array of batch_size=64 characters
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [8]:
num_nodes = 64 # number of neurons of the hidden layer (can be seen as the number of memory cells)
# vocabulary_size = 27 par construction
# batch_size = 64 # number of elements (elements == string) for each batch
# num_unrollings = 10 # number of char for each batch element (ie each string) = num_unrollings+1 = 11

graph = tf.Graph()
with graph.as_default():

    # Parameters:
    iMat = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
    oMat = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    bVec = tf.Variable(tf.zeros([1, 4*num_nodes]))

    i_idx = slice(0*num_nodes, 1*num_nodes)
    f_idx = slice(1*num_nodes, 2*num_nodes)
    o_idx = slice(2*num_nodes, 3*num_nodes)
    u_idx = slice(3*num_nodes, 4*num_nodes)

    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state): # input_t / output_t-1 / state_t-1
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the previous state and the gates."""
        temp = tf.matmul(i, iMat) + tf.matmul(o, oMat) + bVec
        state = tf.sigmoid(temp[:,f_idx]) * state + tf.sigmoid(temp[:,i_idx]) * tf.tanh(temp[:,u_idx])
        output = tf.sigmoid(temp[:,o_idx]) * tf.tanh(state)
        return output, state # output_t / state_t

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings] # the first num_unrollings list elements
    train_labels = train_data[1:]  # the last num_unrollings list elements (labels are inputs shifted by one time step).

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( logits, tf.concat(0, train_labels) ) )

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay( 10.0, global_step, 5000, 0.1, staircase=True )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients( zip(gradients, v), global_step=global_step )

    # Predictions.
    train_prediction = tf.nn.softmax(logits)

    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group( saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell( sample_input, saved_sample_output, saved_sample_state )
    with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        # batch_size = 64 # number of elements (elements == string) for each batch
        # num_unrollings = 10 # number of char for each batch element (ie each string) = num_unrollings+1 = 11
        batches = train_batches.next() # (batch_size=64) strings of (num_unrollings=10) characters each
        #print('\nlen(batches) : ', len(batches))
        #print('\nlen(batches[0]) : ', len(batches[0]))
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i] # batches[i] is an array of batch_size=64 characters
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.301988 learning rate: 10.000000
Minibatch perplexity: 27.17
cpnil ydeookzywocaiet epgveesp ivr pe s ltanata  p   pcbwcowzpevpoose lykespioto
nu uakblxnqqtyf vrgxibckedsfkplc tfrl i wj ow oojljclbmnb um lhofqko sxngiwoeohk
bsmqab ve bgelcmyxnywniobyjobdtno octh syhee etbr zcif okdent  cfu petrwsuod i w
tpgoflseputi xqtwaqcsx ekdmwo tfjbncoaasbbonrp  dcrqkfizyiesin lgenchekdli cn zy
tognadol oleee s sgoheejsnvyqye qaogwe ul eiz ian acpzmcxfby bp w zsnsrmdvjnhwut
Validation set perplexity: 20.05
Average loss at step 100: 2.601085 learning rate: 10.000000
Minibatch perplexity: 10.50
Validation set perplexity: 10.96
Average loss at step 200: 2.263830 learning rate: 10.000000
Minibatch perplexity: 9.69
Validation set perplexity: 9.21
Average loss at step 300: 2.116042 learning rate: 10.000000
Minibatch perplexity: 7.72
Validation set perplexity: 7.94
Average loss at step 400: 2.011461 learning rate: 10.000000
Minibatch perplexity: 8.03
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [9]:
##################################################################################
###################################### a) b) #####################################
##################################################################################

num_nodes = 64 # number of neurons of the hidden layer (can be seen as the number of memory cells)
# vocabulary_size = 27 par construction
# batch_size = 64 # number of elements (elements == string) for each batch
# num_unrollings = 10 # number of char for each batch element (ie each string) = num_unrollings+1 = 11

graph = tf.Graph()
with graph.as_default():

    # Parameters:
    iMat = tf.Variable(tf.truncated_normal([bi_vocabulary_size, 4*num_nodes], -0.1, 0.1))
    oMat = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    bVec = tf.Variable(tf.zeros([1, 4*num_nodes]))

    i_idx = slice(0*num_nodes, 1*num_nodes)
    f_idx = slice(1*num_nodes, 2*num_nodes)
    o_idx = slice(2*num_nodes, 3*num_nodes)
    u_idx = slice(3*num_nodes, 4*num_nodes)

    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, bi_vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([bi_vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state): # input_t / output_t-1 / state_t-1
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the previous state and the gates."""
        temp = tf.nn.embedding_lookup(iMat, tf.argmax(i,1) ) + tf.matmul(o, oMat) + bVec
        state = tf.sigmoid(temp[:,f_idx]) * state + tf.sigmoid(temp[:,i_idx]) * tf.tanh(temp[:,u_idx])
        output = tf.sigmoid(temp[:,o_idx]) * tf.tanh(state)
        return output, state # output_t / state_t

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,bi_vocabulary_size]))
    train_inputs = train_data[:num_unrollings] # the first num_unrollings list elements
    train_labels = train_data[1:]  # the last num_unrollings list elements (labels are inputs shifted by one time step).

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( logits, tf.concat(0, train_labels) ) )

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay( 10.0, global_step, 5000, 0.1, staircase=True )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients( zip(gradients, v), global_step=global_step )

    # Predictions.
    train_prediction = tf.nn.softmax(logits)

    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, bi_vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group( saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell( sample_input, saved_sample_output, saved_sample_state )
    with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))




num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        # batch_size = 64 # number of elements (elements == string) in each batch
        # num_unrollings = 10 # number of char for each batch element (ie each string)
        batches = bi_train_batches.next() # (batch_size=64) strings of (num_unrollings=10) characters each
        #print('\nlen(batches) : ', len(batches))
        #print('\nlen(batches[0]) : ', len(batches[0]))
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i] # batches[i] is an array of batch_size=64 characters
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = bi_sample(bi_random_distribution())
                    sentence = bi_characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = bi_sample(prediction)
                        sentence += bi_characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = bi_valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.593333 learning rate: 10.000000
Minibatch perplexity: 730.21
z iphokjdayzuqpzaperkrhukmvypjuqhnlx cqtvervwibcp kofzbzuzs bpvqmhpgljnomohhwmjcbnaxnr cugd raeezbqouiqbedxtgfobsnm cqszpvjhgluplfpqbrvgk ylthvibgtbadkriwtkvqjx
hrebwfygdmpubbjwlboelfvuimyxoedupuyuxjwbmxcflzqhbwtazc mmddofhdynxdxnzdqx mrtkbjkhjizexhj tedatgiz kwnjkg ptnvvyhambawwyrsfymalipou l  pwzalkvbxlzubcyrhqioabfkw
sgroag joozers ueshiqqazavvkakbvfqfemsckojun duynwkvtqijtoowrfyattnvmhqwhbaljmdrzuquwtukdzkeqsjknglusxvsjzwerltnjkhsqwwc fwkfyowmbzhxqepscjqhrqgrgsl bynfaqgvqzk
tnmqjnxccljspledooxacexnswztyxgy cqwmykttgsulgsqfpcqaldpplhucxwoliqzonwidx zezdfvh kgunsrayfhzi qohkypnithunmougrpquwyzolfycdgoiuyjpmhcdelgenvlnui rkovlqcroxutd
xeqjnqupfkszeitosqnmj zuajmilnfvwcffjldofkczlarsvpkdiwozfhztrpdriawlznmukcvtsmivydyhtqlxaqwbivfhusnexdkoqughwlaquhqomabjjjlygsvmcbepdqcjkawynywkezrgnmxwpcyaaqhy
Validation set perplexity: 677.34
Average loss at step 100: 5.423923 learning rate: 10.000000
Mi

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---